## Stride Detector(SD), 包含以下字段：
- PC（程序计数器）: 48比特。用于记录加载指令的地址，作为唯一标识。
- Previous Address（前一个地址）: 48比特。存储该指令上一次的访问地址。
- Stride（步长）: 8比特。存储当前的步长值，用于判断步长模式。
- Last Prefetch（上次预取地址）: 48比特。记录上次生成的预取地址，用于防止重复预取。
- Seen（是否检测到）: 1比特。标记该指令是否已经检测到步长模式。
- LIL（最后一个间接加载）: 16比特。记录最后一个间接加载的地址偏移量。
- Stride Confidence（步长置信度）: 2比特。表示步长模式检测的置信度（如：0表示未检测到，1表示低置信度，2表示中置信度，3表示高置信度）。
- LIL Confidence（LIL置信度）: 2比特。表示最后一个间接加载检测的置信度。

In [13]:
class StrideDetectorEntry:
    def __init__(self, pc):
        self.pc = pc  ###### 48-bit 程序计数器
        self.previous_address = 0  ###### 48-bit 前一个地址
        self.stride = 0  ###### 8-bit 步长
        self.last_prefetch = 0  ###### 48-bit 上次预取地址
        self.seen = 0  ###### 1-bit 是否检测到（标记该PC的间接加载模式）
        self.lil = 0  ###### 16-bit 最后一个间接加载
        self.stride_confidence = 0  ###### 2-bit 步长置信度
        self.lil_confidence = 0  ###### 2-bit LIL置信度

    def set_lil(self, lil):
        """设置LIL和LIL置信度"""
        if lil == self.lil:
            self.lil_confidence = min(self.lil_confidence + 1, 3)  # 增加置信度
        else:
            self.lil = lil
            self.lil_confidence = 1  # 重置置信度
            self.reset_seen()  # LIL发生变化时，重置Seen状态

    def set_stride(self, stride):
        """设置Stride值及其置信度"""
        if stride == self.stride:
            self.stride_confidence = min(self.stride_confidence + 1, 3)  # 增加置信度
        else:
            self.stride = stride
            self.stride_confidence = 1  # 重置置信度为1（新步长被发现）
            self.reset_seen()  # 步长发生变化时，重置Seen状态

    def set_last_prefetch(self, prefetch_addr):
        """设置LastPrefetch地址"""
        self.last_prefetch = prefetch_addr

    def set_seen(self):
        """设置该PC的间接加载模式已被检测到"""
        self.seen = 1

    def reset_seen(self):
        """重置Seen状态"""
        self.seen = 0

    def __repr__(self):
        return (f"PC: {hex(self.pc)}, PrevAddr: {self.previous_address}, Stride: {self.stride}, "
                f"LastPrefetch: {self.last_prefetch}, Seen: {self.seen}, LIL: {self.lil}, "
                f"StrideConf: {self.stride_confidence}, LILConf: {self.lil_confidence}")


class StrideDetector:
    def __init__(self, max_entries=16):
        self.entries = {}  ###### 存储步长检测的条目
        self.max_entries = max_entries  ###### 最大条目数
        self.access_order = []  ###### 记录访问顺序，用于LRU替换策略

    def _update_access_order(self, pc):
        """ 更新访问顺序，移到最后表示最近使用 """
        if pc in self.access_order:
            self.access_order.remove(pc)
        self.access_order.append(pc)

    def _evict_entry(self):
        """ 使用LRU策略移除最早访问的条目 """
        if len(self.access_order) > 0:
            oldest_pc = self.access_order.pop(0)  ###### 移除最久未使用的条目
            del self.entries[oldest_pc]
            print(f"Evicted PC {hex(oldest_pc)} due to capacity limit.")

    def detect_stride(self, pc, current_address, indirect_address=None):
        """ 检测步长模式，并更新步长置信度、LastPrefetch、Seen字段 """
        ###### 如果没有该PC的记录，则初始化一个条目
        if pc not in self.entries:
            ###### 如果当前条目数超过最大限制，执行替换策略
            if len(self.entries) >= self.max_entries:
                self._evict_entry()
            
            ###### 创建新的Stride Detector条目
            self.entries[pc] = StrideDetectorEntry(pc)
        
        entry = self.entries[pc]
        stride_detected = False

        ###### 计算当前步长
        stride = current_address - entry.previous_address

        ###### 更新步长置信度和步长值
        entry.set_stride(stride)

        ###### 当步长置信度达到1时，认为检测到步长模式（Stride Detected）
        if entry.stride_confidence >= 1:
            stride_detected = True

        ###### 更新条目中的前一个地址
        entry.previous_address = current_address

        ###### 如果存在间接加载地址，则更新LIL
        if indirect_address is not None:
            entry.set_lil(indirect_address)

        ###### 判断是否需要进行预取，依据Seen和LastPrefetch
        if entry.seen == 0 or current_address != entry.last_prefetch:
            # 进行预取，并更新LastPrefetch
            print(f"Prefetching address: {current_address}")
            entry.set_last_prefetch(current_address)
            entry.set_seen()  # 标记该PC已被检测到

        ###### 当步长置信度降低或模式变化时，重置Seen字段
        if not stride_detected:
            entry.reset_seen()

        ###### 更新访问顺序
        self._update_access_order(pc)

        return stride_detected, entry.stride_confidence, entry.lil, entry.lil_confidence, entry.last_prefetch, entry.seen

    def __repr__(self):
        entries_repr = "\n".join([str(entry) for entry in self.entries.values()])
        return f"Stride Detector Entries (Max Entries: {self.max_entries}):\n{entries_repr}"


if __name__ == "__main__":
    sd = StrideDetector(max_entries=3)

    
    test_cases = [
        # PC = 0x1, 初始步长与LIL测试
        (0x1, 1000, None),      # 初始PC = 0x1, 地址 = 1000
        (0x1, 1016, 3000),      # 间接加载1：LIL = 3000，预取1016，Seen标记
        (0x1, 1032, 3000),      # 间接加载2：LIL相同，LIL置信度提升，预取1032
        (0x1, 1032, 3000),      # 重复预取地址：不进行预取操作
        (0x1, 1048, 3016),      # LIL发生变化：预取1048，重置Seen

        # PC = 0x2, 不同步长和LIL的测试
        (0x2, 2000, None),      # 新PC = 0x2, 地址 = 2000，无间接加载，首次预取
        (0x2, 2016, 4000),      # 间接加载1：LIL = 4000，预取2016
        (0x2, 2032, 4016),      # LIL更新为4016，预取2032，重置Seen
        (0x2, 2032, 4016),      # 重复地址：不进行预取
        (0x2, 2048, 4016),      # 新地址：预取2048

        # PC = 0x3, 混合直接和间接加载
        (0x3, 3000, None),      # 新PC = 0x3, 地址 = 3000，无间接加载
        (0x3, 3016, 5000),      # 间接加载1：LIL = 5000，预取3016
        (0x3, 3032, 5000),      # 间接加载2：LIL相同，LIL置信度提升，预取3032
        (0x3, 3050, 5016),      # LIL发生变化，重置Seen

        # 新PC = 0xf, 触发LRU替换策略
        (0xf, 4000, None),      # 新PC = 0xf, 地址 = 4000（触发LRU替换）
        (0xf, 4016, 6000),      # 间接加载1：LIL = 6000，预取4016
        (0xf, 4032, 6000),      # 间接加载2：LIL相同，LIL置信度提升，预取4032
        (0x3, 3050, 5016),      # 回到PC = 0x3, LIL = 5016（验证LRU替换效果）
    ]

    print("Initial State:")
    print(sd)

    ###### 对每个测试用例进行步长检测
    for pc, addr, lil_addr in test_cases:
        stride_detected, stride_conf, lil, lil_conf, last_prefetch, seen = sd.detect_stride(pc, addr, lil_addr)
        print(f"PC: {hex(pc)}, Addr: {addr}, LIL: {lil}, LastPrefetch: {last_prefetch} -> "
              f"Stride Detected: {stride_detected}, Stride Confidence: {stride_conf}, "
              f"LIL Confidence: {lil_conf}, Seen: {seen}")
        print(f"{sd}\n")

    ###### 打印最终的Stride Detector状态
    print("Final State:")
    print(sd)


Initial State:
Stride Detector Entries (Max Entries: 3):

Prefetching address: 1000
PC: 0x1, Addr: 1000, LIL: 0, LastPrefetch: 1000 -> Stride Detected: True, Stride Confidence: 1, LIL Confidence: 0, Seen: 1
Stride Detector Entries (Max Entries: 3):
PC: 0x1, PrevAddr: 1000, Stride: 1000, LastPrefetch: 1000, Seen: 1, LIL: 0, StrideConf: 1, LILConf: 0

Prefetching address: 1016
PC: 0x1, Addr: 1016, LIL: 3000, LastPrefetch: 1016 -> Stride Detected: True, Stride Confidence: 1, LIL Confidence: 1, Seen: 1
Stride Detector Entries (Max Entries: 3):
PC: 0x1, PrevAddr: 1016, Stride: 16, LastPrefetch: 1016, Seen: 1, LIL: 3000, StrideConf: 1, LILConf: 1

Prefetching address: 1032
PC: 0x1, Addr: 1032, LIL: 3000, LastPrefetch: 1032 -> Stride Detected: True, Stride Confidence: 2, LIL Confidence: 2, Seen: 1
Stride Detector Entries (Max Entries: 3):
PC: 0x1, PrevAddr: 1032, Stride: 16, LastPrefetch: 1032, Seen: 1, LIL: 3000, StrideConf: 2, LILConf: 2

Prefetching address: 1032
PC: 0x1, Addr: 1032, LIL: 